# Automated ML

In this capstone project I will showcase how we can use data science as a investigation tool, here we use classfication algorithm to distinguish between normal traffic (good connections) and intrusion or attacks traffic (bad connections). A connection is a sequence of TCP packets starting and ending at some well difined times, between which data flows to and from source IP address to a target IP address under some well defined protocol. We will create Intrusion Detection System (IDS)

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

#### Import Dependencies.
In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import os

## Dataset

### Data description
Data is collected by packet analyzers (also known as packet/network/protocol snifers) intercept and log traffic in the network.The dataset that we will use is the NSLKDD dataset. The original 1999 KDD Cup dataset was created for the DARPA Intrusion Detection Evaluation Program, prepared and managed by MIT Lincoln Laboratory. The data was collected over nine
weeks and consists of raw tcpdump traffic in a local area network (LAN) that simulates the environment of a typical United States Air Force LAN. Some network attacks were deliberately carried out during the recording period. There were 38 different
types of attacks, but only 24 are available in the training set. 

#### Data references

https://www.unb.ca/cic/datasets/nsl.html     
https://www.kaggle.com/hassan06/nslkdd

## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [ ]:
from azureml.core import Workspace, Experiment

vrk_auto_ids_ws = Workspace.from_config()
vrk_auto_ids_exp = Experiment(workspace=vrk_auto_ids_ws, name="vrk_ids_auto_exp")

print('Workspace name: ' + vrk_auto_ids_ws.name, 
      'Azure region: ' + vrk_auto_ids_ws.location, 
      'Subscription id: ' + vrk_auto_ids_ws.subscription_id, 
      'Resource group: ' + vrk_auto_ids_ws.resource_group, sep = '\n')

auto_ids_run = vrk_auto_ids_exp.start_logging()

### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your run. 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=vrk_ids_ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(vrk_ids_ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

### Download and prepare data for AutoML learning


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory

kdd_auto_webpath = [
                            'https://raw.githubusercontent.com/venkataravikumaralladi/AzureMLCapstoneProject/main/KDDTrain.csv'
                            ]

#create bankmarketing data set in tabular format using TabularDatasetFactory
ids_auto_dataset = TabularDatasetFactory.from_delimited_files(path=kdd_auto_webpath)

In [ ]:
# class variables
network_data_column_names = [ 
                  'duration', 'protocol_type', 'service',
                  'flag', 'src_bytes', 'dst_bytes',
                  'land', 'wrong_fragment', 'urgent',
    
            
                  'hot', 'num_failed_logins', 'logged_in',
                  'num_compromised', 'root_shell', 'su_attempted',
                  'num_root', 'num_file_creations', 'num_shells',
                  'num_access_files', 'num_outbound_cmds', 'is_hot_login',
                  'is_guest_login',
    
                 
                  'count', 'srv_count', 'serror_rate',
                  'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
                  'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
                 
                  'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
                  'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
                  'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
                  'dst_host_srv_rerror_rate',
    
                   'attack_type',
                   'success_pred' ]

train_df = ids_auto_dataset.to_pandas_dataframe().dropna()
train_df.columns = network_data_column_names

# For this analysis we drop "success_pred" column
train_df.drop('success_pred', axis=1, inplace=True)
    
# Drop attack type in training data which is to be predicted.
train_X = train_df.drop("attack_type", axis=1)
train_Y = train_df['attack_type']

from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ids_auto_dataset)
x = x.join(y)
x.head()

#store data frame to data store for AutoMLConfig training
x.to_csv('training/bank_training_data.csv')
default_datastore = vrk_bank_ws.get_default_datastore()
default_datastore

#upload training data to data store for AutomMLConfig training
default_datastore.upload(src_dir='training', target_path='data/')

# get data set pointer to data store for bank training data
from azureml.core import Dataset
banktraining_dataset = Dataset.Tabular.from_delimited_files(path=[(default_datastore, ('data/bank_training_data.csv'))])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
vrk_bnk_automl_config = AutoMLConfig(
                   experiment_timeout_minutes=30,
                   task="classification",
                   primary_metric="accuracy",
                   compute_target = cpu_cluster,
                   training_data=banktraining_dataset,
                   label_column_name='y',
                   n_cross_validations=5)

In [ ]:
# Submit your automl run
from azureml.pipeline.steps import AutoMLStep
vrk_auto_bank_exp = Experiment(workspace=vrk_bank_ws, name="vrk_auto_bank_train_exp")
automl_bnk_run = vrk_auto_bank_exp.submit(vrk_bnk_automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(automl_bnk_run).show()

In [ ]:
automl_bnk_run.wait_for_completion()

In [ ]:
assert(automl_bnk_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
best_auto_run, best_fitted_model = automl_bnk_run.get_output()

In [ ]:
#TODO: Save the best model
best_fitted_model._final_estimator


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_auto_run.register_model(model_name='vrk_bank_best_auto_model_predictor', model_path='./outputs')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service